In [5]:
from langchain import ConversationChain,PromptTemplate
import torch
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes import  VectorstoreIndexCreator
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.llms import GPT4All

import os
from langchain.memory import VectorStoreRetrieverMemory
from langchain.callbacks import get_openai_callback

import streamlit as st
from streamlit_chat import message
from dotenv import load_dotenv

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


In [6]:
loader = DirectoryLoader('D:/OneDrive/Documents/Obsidian/Projects/myVault/', glob="**/*.md",recursive=True, show_progress=True, use_multithreading=True, loader_cls=TextLoader)
docs = loader.load()
len(docs)

100%|██████████| 34/34 [00:00<00:00, 767.90it/s]


34

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [15]:
llm = GPT4All(model="./ggml-mpt-7b-instruct.bin", top_p=0.92, top_k=0,  temp=0.5, repeat_penalty=1.1, n_threads=12, n_batch=16)

Found model file.


In [9]:
index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])
retriever = index.vectorstore.as_retriever(search_kwargs=dict(k=2))
memory = VectorStoreRetrieverMemory(retriever=retriever)


100%|██████████| 34/34 [00:00<00:00, 1888.68it/s]


In [10]:
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
TEMPLATE = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

In [11]:
_DEFAULT_TEMPLATE = """
Below is an instruction that describes a task. Write a response that appropriately completes the request.
###Instruction: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
Do not make up answers and provide only information that you have.
Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)
{input}

### Response:
"""

In [12]:
PROMPT = PromptTemplate(
    input_variables=[ "history", "input"], template=_DEFAULT_TEMPLATE
)


In [17]:
import torch


conversation_with_summary = ConversationChain(
    llm=llm, 
    prompt=PROMPT,
    # We set a very low max_token_limit for the purposes of testing.
    memory = memory,
    verbose=True
    )


with torch.inference_mode():

   conversation_with_summary.predict(input = "what are my social media handles")



> Entering new ConversationChain chain...
Prompt after formatting:

Below is an instruction that describes a task. Write a response that appropriately completes the request.
###Instruction: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
Do not make up answers and provide only information that you have.
Relevant pieces of previous conversation:
input: what are my social media handles
response: `ris:Facebook`: [https://www.facebook.com/arbaballa](https:// www.facebook.com/arbaBallaL/)`ris:Linkedin`: https://www.linkedi n. com/in/akshay-balla l/`ris:GitHub`:  https://github.com/akshayballal95`ris:Twitter`: @aksheybalal95
input: what are my social media handles
response: ris:Facebook`: [https://www.facebook. com/arbaballa](https:// www.facebook.

(You do not need to use these pieces of information if not re